In [1]:
import torch
import numpy as np
import genesis as gs

from scipy.spatial.transform import Rotation as R

[I 12/28/24 14:55:12.888 18826] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout


In [2]:
########################## init ##########################
gs.init(backend=gs.gpu, logging_level="error")

In [3]:
file_path = '../../genesis/assets/xml/x1/mjcf/xyber_x1_flat.xml'
# file_path = 'xml/franka_emika_panda/panda.xml'

In [ ]:
########################## create a scene and entities ##########################
scene = gs.Scene(show_viewer=True, show_FPS=False)
plane = scene.add_entity(gs.morphs.Plane())
robot = scene.add_entity(
    gs.morphs.MJCF(file  = file_path,
                   pos   = (0.0, 0.0, 0.65),
                   euler = (0, 0, 0),),
)
box = scene.add_entity(gs.morphs.Box(pos   = (0.3, 0.0, 1.0),
                                     size  = (0.2, 0.2, 0.2))
                                     fixed = True,)

########################## build ##########################
scene.build()
state = scene.get_state()

In [5]:
robot

────────────────────── <gs.RigidEntity> ──────────────────────
                'n_qs': <int>: 14
              'n_dofs': <int>: 14
             'n_links': <int>: 30
             'n_geoms': <int>: 10
             'n_cells': <numpy.int64>: 1129172
             'n_verts': <int>: 1748
             'n_faces': <int>: 3456
             'n_edges': <int>: 5184
            'n_joints': <int>: 30
            'n_vgeoms': <int>: 38
            'n_vverts': <int>: 379162
            'n_vfaces': <int>: 769092
               'q_end': <int>: 14
             'q_start': <int>: 0
            'is_built': <bool>: True
             'dof_end': <int>: 14
           'dof_start': <int>: 0
                 'idx': <int>: 1
                 'sim': <gs.Simulator>
                 'uid': <gs.UID>('f47b7a4-a966145abb0bd8199b544637d')
           'base_link': <gs.RigidLink>: <1ec39d8>, name: 'x1-body', idx: 1
          'base_joint': <gs.RigidJoint>: <5bd7a90>, name: 'x1-body_joint', idx: 1, type: <FIXED: 0>
       'base_l

In [6]:
# x1-body floating_base <FREE: 4>: dof 6
# arms dof: 14
# legs dof: 12 --> 0

print(f'number of links: {robot.n_links}')
for link in robot.links:
    print(f'{link.idx} {link.name} {link.joint.name} {link.joint.type}')

number of links: 30
1 x1-body x1-body_joint <FIXED: 0>
2 body_yaw body_yaw_joint <FIXED: 0>
3 left_hip_pitch_link left_hip_pitch_link_joint <FIXED: 0>
4 right_hip_pitch_link right_hip_pitch_link_joint <FIXED: 0>
5 body_roll body_roll_joint <FIXED: 0>
6 left_hip_roll_link left_hip_roll_link_joint <FIXED: 0>
7 right_hip_roll_link right_hip_roll_link_joint <FIXED: 0>
8 body_pitch body_pitch_joint <FIXED: 0>
9 left_hip_yaw_link left_hip_yaw_link_joint <FIXED: 0>
10 right_hip_yaw_link right_hip_yaw_link_joint <FIXED: 0>
11 left_shoulder_pitch_link left_shoulder_pitch <REVOLUTE: 1>
12 right_shoulder_pitch_link right_shoulder_pitch <REVOLUTE: 1>
13 left_knee_pitch_link left_knee_pitch_link_joint <FIXED: 0>
14 right_knee_pitch_link right_knee_pitch_link_joint <FIXED: 0>
15 left_shoulder_roll_link left_shoulder_roll <REVOLUTE: 1>
16 right_shoulder_roll_link right_shoulder_roll <REVOLUTE: 1>
17 left_ankle_pitch_link left_ankle_pitch_link_joint <FIXED: 0>
18 right_ankle_pitch_link right_ankle_pit

In [7]:
########################## joints ##########################
leg_joint_names = ['left_hip_pitch', 'left_hip_roll', 'left_hip_yaw', 'left_knee_pitch',
             'left_ankle_pitch', 'left_ankle_roll', 'right_hip_pitch', 'right_hip_roll',
             'right_hip_yaw', 'right_knee_pitch', 'right_ankle_pitch', 'right_ankle_roll']
arm_joint_names = ['left_shoulder_pitch', 'left_shoulder_roll', 'left_shoulder_yaw', 'left_elbow_pitch',
                   'left_elbow_yaw', 'left_wrist_pitch', 'left_wrist_roll',
                   'right_shoulder_pitch', 'right_shoulder_roll', 'right_shoulder_yaw', 'right_elbow_pitch',
                   'right_elbow_yaw', 'right_wrist_pitch', 'right_wrist_roll']

arm_dofs_idx, leg_dofs_idx, base_dofs_idx = [], [], []

try:
    arm_dofs_idx = [robot.get_joint(name).dof_idx_local for name in arm_joint_names]
    print(f'arm_dofs_idx: {arm_dofs_idx}')
except Exception as e:
    print(e)
    
try:
    leg_dofs_idx = [robot.get_joint(name).dof_idx_local for name in leg_joint_names]
    print(f'leg_dofs_idx: {leg_dofs_idx}')
except Exception as e:
    print(e)

try:
    base_dofs_idx = [robot.get_joint('floating_base').dof_idx_local]
    print(f'base_dofs_idx: {base_dofs_idx}')
except Exception as e:
    print(e)

arm_dofs_idx: [0, 2, 4, 6, 8, 10, 12, 1, 3, 5, 7, 9, 11, 13]
Joint not found for name: left_hip_pitch.
Joint not found for name: floating_base.


In [8]:
########################## links ##########################

# link的索引从1开始
body_link_names = ['x1-body', 'body_yaw', 'body_roll', 'body_pitch']
arm_link_names = [name + '_link' for name in arm_joint_names]
leg_link_names = [name + '_link' for name in leg_joint_names]
link_names = body_link_names + arm_link_names + leg_link_names

info = []
for name in link_names:
    idx = robot.get_link(name).idx
    pos = robot.get_links_pos()[idx-1]
    info.append((idx, name, pos))

info.sort(key=lambda x: x[0])
for idx, name, pos in info:
    print(f'{idx} {name} {pos}')

1 x1-body tensor([0.0000, 0.0000, 0.6500], device='cuda:0')
2 body_yaw tensor([0.0025, 0.0000, 0.7655], device='cuda:0')
3 left_hip_pitch_link tensor([0.0025, 0.0923, 0.6379], device='cuda:0')
4 right_hip_pitch_link tensor([ 0.0025, -0.0923,  0.6379], device='cuda:0')
5 body_roll tensor([0.0025, 0.0000, 0.8060], device='cuda:0')
6 left_hip_roll_link tensor([0.0429, 0.1339, 0.5962], device='cuda:0')
7 right_hip_roll_link tensor([ 0.0429, -0.1339,  0.5962], device='cuda:0')
8 body_pitch tensor([0.0025, 0.0000, 0.8060], device='cuda:0')
9 left_hip_yaw_link tensor([0.0023, 0.1339, 0.5124], device='cuda:0')
10 right_hip_yaw_link tensor([ 0.0023, -0.1339,  0.5185], device='cuda:0')
11 left_shoulder_pitch_link tensor([0.0025, 0.1458, 1.0620], device='cuda:0')
12 right_shoulder_pitch_link tensor([ 0.0024, -0.1458,  1.0620], device='cuda:0')
13 left_knee_pitch_link tensor([0.0023, 0.1002, 0.3702], device='cuda:0')
14 right_knee_pitch_link tensor([ 0.0023, -0.1004,  0.3702], device='cuda:0')
15 

In [9]:
############ 可选：设置控制增益 ############
# 设置位置增益
robot.set_dofs_kp(
    kp             = np.ones(robot.n_dofs) * 40,
    # dofs_idx_local = arm_dofs_idx,
)
# 设置速度增益
robot.set_dofs_kv(
    kv             = np.ones(robot.n_dofs) * 3.0,
    # dofs_idx_local = arm_dofs_idx,
)
# 设置安全的力范围
robot.set_dofs_force_range(
    lower          = np.ones(robot.n_dofs) * -100,
    upper          = np.ones(robot.n_dofs) * 100,
    # dofs_idx_local = arm_dofs_idx,
)
kv = robot.get_dofs_kv()
print(f'kv: {kv}')
kp = robot.get_dofs_kp()
print(f'kp: {kp}')

kv: tensor([3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.],
       device='cuda:0')
kp: tensor([40., 40., 40., 40., 40., 40., 40., 40., 40., 40., 40., 40., 40., 40.],
       device='cuda:0')


In [10]:
if len(base_dofs_idx) > 0:
    robot.get_joint('floating_base')

In [11]:
########################## end effectors ##########################
end_effector_names = ['left_wrist_roll_link', 'right_wrist_roll_link']
effector_pos = robot.get_links_pos()
end_effector_idx = [robot.get_link(name).idx_local for name in end_effector_names]
ee_pos = [effector_pos[i] for i in end_effector_idx]
print(f'end effector positions: {ee_pos}')

end effector positions: [tensor([0.0026, 0.1990, 0.6439], device='cuda:0'), tensor([ 0.0023, -0.1990,  0.6439], device='cuda:0')]


In [12]:
########################## inverse_kinematics ##########################
ee_link = [robot.get_link(name) for name in end_effector_names]
qpos = robot.inverse_kinematics_multilink(ee_link, [[0.1, 0.1, 0.6], [ 0.3, -0.3,  0.5]])

In [21]:
default_qpos = torch.zeros(qpos.size(), device=qpos.device)
robot.set_dofs_position(default_qpos)
for i in range(100):
    scene.step()

In [18]:
# 规划运动路径
path = robot.plan_path(
    qpos_goal     = qpos,
    num_waypoints = int(2 / scene.dt), # 2秒时长
)

# 执行规划路径
for waypoint in path:
    robot.control_dofs_position(waypoint)
    scene.step()

# 等待到达最后一个路径点
for i in range(100):
    scene.step()

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def quaternion_to_euler(quaternion):
    """
    将四元数转换为欧拉角（单位：弧度）。
    参数：
    quaternion (array-like): 四元数 [x, y, z, w]
    返回：
    euler_angles (array): 对应的欧拉角 [roll, pitch, yaw]（单位：弧度）
    """
    # 创建旋转对象
    r = R.from_quat(quaternion)
    # 获取欧拉角 (yaw, pitch, roll)，使用 ypr 顺序
    euler_angles = r.as_euler('zyx', degrees=False)
    return torch.tensor(euler_angles, device=device)

quaternion = [1, 0, 0,  1.0000e+00]  # 例如一个四元数
euler_angles = quaternion_to_euler(quaternion)
print(euler_angles)


tensor([0.0000, 0.0000, 1.5708], device='cuda:0', dtype=torch.float64)


In [15]:
origin_qpos = robot.get_qpos()
print(f'origin_qpos: {origin_qpos}')
arm_dof_pos = robot.get_dofs_position(arm_dofs_idx)
print(f'arm_dof_pos: {arm_dof_pos}')
if len(base_dofs_idx) > 0:
    origin_dof_pos = robot.get_dofs_position(base_dofs_idx[0])
    print(f'origin_dof_pos: {origin_dof_pos}')
    origin_q_euler = quaternion_to_euler(origin_qpos[3:7].cpu())
    print(f'origin_q_euler: {origin_q_euler}')

origin_qpos: tensor([-3.1401, -0.3861,  3.1400,  0.1683, -2.5539,  0.0560,  0.1030, -0.0310,
        -1.6311,  2.1473, -2.1143, -1.1285, -2.1733, -2.2354], device='cuda:0')
arm_dof_pos: tensor([-3.1401,  3.1400, -2.5539,  0.1030, -1.6311, -2.1143, -2.1733, -0.3861,
         0.1683,  0.0560, -0.0310,  2.1473, -1.1285, -2.2354], device='cuda:0')


In [16]:
# 通过set_dofs_position暴力执行
if len(base_dofs_idx) > 0:
    q_euler = quaternion_to_euler(qpos[3:7])
    qpos_euler = torch.cat([qpos[:3], q_euler, qpos[7:]])
else:
    qpos_euler = qpos
robot.set_dofs_position(qpos_euler)

if len(base_dofs_idx) > 0:
    dof_euler = robot.get_dofs_position(base_dofs_idx[0])[3:]
    print(f'q_euler: {q_euler}')
    print(f'dof_euler: {dof_euler}')
print(f'qpos: {qpos}')
print(f'dofs_position: {robot.get_dofs_position(arm_dofs_idx)}')

qpos: tensor([-3.1400, -0.4460,  3.1400,  0.1906, -2.5539,  0.0464,  0.1082, -0.0521,
        -1.6309,  2.1479, -2.1122, -1.1287, -2.1721, -2.2374], device='cuda:0')
dofs_position: tensor([-3.1400,  3.1400, -2.5539,  0.1082, -1.6309, -2.1122, -2.1721, -0.4460,
         0.1906,  0.0464, -0.0521,  2.1479, -1.1287, -2.2374], device='cuda:0')


In [17]:
for i in range(robot.n_dofs):
    if i in arm_dofs_idx:
        print(f'{i} arm {qpos[i+1]}')
    elif i in leg_dofs_idx:
        print(f'{i} leg {qpos[i+1]}')
    else:
        print(f'{i} base {qpos[i+1]}')

0 arm -0.44601649045944214
1 arm 3.140000104904175
2 arm 0.1906452178955078
3 arm -2.5539438724517822
4 arm 0.04636955261230469
5 arm 0.10824096202850342
6 arm -0.052115052938461304
7 arm -1.630906343460083
8 arm 2.1478958129882812
9 arm -2.1121647357940674
10 arm -1.128735065460205
11 arm -2.1721153259277344
12 arm -2.2374267578125


IndexError: index 14 is out of bounds for dimension 0 with size 14